In [ ]:
#Temperatur dependent defect document
# General
import numpy as np
import matplotlib.pyplot as plt
from eskild_function import *

# For handling structures and visualizing structures
from ase import Atoms
from ase.build import graphene_nanoribbon
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from ase.io import read, write

# For MD
from ase.calculators.tersoff import Tersoff
from ase.constraints import FixAtoms
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.langevin import Langevin
from ase import units
from ase.neighborlist import neighbor_list

import os
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

In [ ]:
# --- Structure Definition ---
kind = "armchair"     # "armchair" or "zigzag"
n = 5                 # width parameter
length = 1            # periodic repetitions along z, i.e., transport direction here
vacuum = 15.0         # vacuum in non-periodic directions (Å)
bond = 1.43877067     # Optimized using C.tersoff potential discussed below

ribbon = graphene_nanoribbon(n=n,
                             m=length,
                             type=kind,
                             C_C=bond,
                             vacuum=vacuum)
ribbon.pbc = True
ribbon = sort_atoms(ribbon) # Function from eskild_function
structure = ribbon

# --- Hamiltonian Setup (Pristine) ---
# Create a transport structure with enough length to define leads
Ntransport = 5 
pristine_structure = structure.repeat((1, 1, Ntransport))
pos = pristine_structure.positions

# Sort atoms (Standard sorting for this project)
x_sort_args = np.lexsort((pos[:, 0], pos[:, 2]))
if len(x_sort_args) > 10:
    x_sort_args[[9, 10]] = x_sort_args[[10, 9]]
pos_sorted = pos[x_sort_args]

# Parameters for splitting
n_lead_atoms = 20 

# Extract Matrices
H_full = hamiltonian(pos_sorted) 

idx_L = np.arange(n_lead_atoms)
idx_R = np.arange(len(pos_sorted)-n_lead_atoms, len(pos_sorted))
idx_D = np.arange(n_lead_atoms, len(pos_sorted)-n_lead_atoms)

H_L = H_full[np.ix_(idx_L, idx_L)]      
H_D = H_full[np.ix_(idx_D, idx_D)]      
H_R = H_full[np.ix_(idx_R, idx_R)]      
t_L = H_full[np.ix_(idx_D[:n_lead_atoms], idx_L)]      
t_R = H_full[np.ix_(idx_D[-n_lead_atoms:], idx_R)]   

# --- MD Setup ---
timestep = 1.0
calc = Tersoff.from_lammps("C.tersoff")

In [ ]:
np.linspace(1,8,8)

In [ ]:
# Define Hamiltonian with larger cutoff to avoid artificial bond breaking
def hamiltonian(xyz):
    bond = 1.43877067
    Vpppi = -2.7
    # Cutoff adjusted to be physically robust:
    # Includes 1st neighbors (~1.42 A) and thermal stretching.
    # Excludes 2nd neighbors (~2.46 A).
    # 2.24 A is a safe middle ground.
    cut = bond + 0.8
    dist = np.linalg.norm(xyz[None, :, :] - xyz[:, None, :], axis=2)
    with np.errstate(divide='ignore', invalid='ignore'):
        H = np.where((dist < cut) & (dist > 0.1), Vpppi * (bond / dist)**2, 0.0)
    return H

def resistance(Transmission):
    # Resistance in units of 1/G0
    # G = G0 * Transmission
    # R = 1/G = 1/(G0 * Transmission) = (1/Transmission) * (1/G0)
    # Returning value in units of 1/G0 means returning 1/Transmission
    # Add small epsilon to avoid division by zero
    T_safe = np.array(Transmission)
    T_safe[T_safe < 1e-12] = 1e-12
    return 1.0 / T_safe

# Parameters needed for the calculation
# Use astype(int) to ensure we have integers for the loop ranges
ntiles_list = np.linspace(5, 50, 45).astype(int) # Varying lengths (number of unit cell repetitions)

# Simulation parameters
md_temp_K = 300.0
md_nsteps = 1000  # Must match the script run on HPC
md_dump = 1       # Must match the script run on HPC
eta = 0.0001
E_fermi_list = [0.1,0.2,0.3,0.4,0.5,1., 2., 3., 4., 5., 6., 7., 8.]  # Must match the script run on HPC

# Pre-compute Self-Energies at Fermi Level for all energies
print("Pre-computing self-energies...")
self_energies = {}
for E in E_fermi_list:
    z_fermi = E + 1j * eta
    sL, _, _, _ = self_energy_decimation(z_fermi, H_L, t_L, iterations=100)
    sR, _, _, _ = self_energy_decimation(z_fermi, H_R, t_R, iterations=100)
    self_energies[E] = (sL, sR)

def calculate_T_at_energy(H_device, sig_L, sig_R, energy):
    """
    Calculate Transmission at specific energy for a given Device Hamiltonian
    """
    size = H_device.shape[0]
    I = np.eye(size)
    z = energy + 1j * eta
    
    # Construct Effective Hamiltonian
    device_zero = np.zeros((size, size), dtype=complex)
    gray_left = device_zero.copy()
    
    n_lead = sig_L.shape[0]
    gray_left[:n_lead, :n_lead] = sig_L
    
    gray_right = device_zero.copy()
    n_lead_R = sig_R.shape[0]
    gray_right[-n_lead_R:, -n_lead_R:] = sig_R
    
    H_eff = H_device + gray_left + gray_right
    
    # Gamma matrices
    gamma_L = 1j * (gray_left - np.conjugate(np.transpose(gray_left)))
    gamma_R = 1j * (gray_right - np.conjugate(np.transpose(gray_right)))
    
    # Green's Function
    G_eff = np.linalg.inv(z*I - H_eff)
    
    T = np.trace(gamma_L @ G_eff @ gamma_R @ np.conjugate(np.transpose(G_eff)))
    return np.real(T)

def run_simulation_for_length(ntile):
    """
    Runs MD and analyzes transport for a specific nanoribbon length.
    Returns: Dictionary {Energy: (avg_resistance_from_avg_T, mean_of_R_samples)}
    """
    # 1. Create Structure for this length
    xyz = structure.positions
    lattice = structure.cell[:]
    tiledir = 2 # z-direction
    
    # Create repeated structure
    for n in range(1, ntile):
        xyz = np.concatenate((xyz, structure.positions + lattice[tiledir, :]*n))
    
    tilemat = np.eye(3, dtype=int)
    tilemat[tiledir, tiledir] = ntile
    lattice_long = tilemat @ lattice
    natoms = len(xyz)
    
    current_md_structure = Atoms(natoms*["C"], positions=xyz, cell=lattice_long, pbc=True)
    
    # Apply Constraints (Leads and Edges)
    natoms_elec = len(structure)
    fixed_uc = 2
    leftinds = list(range(0, natoms_elec*fixed_uc))
    rightinds = list(range(natoms - natoms_elec*fixed_uc, natoms))
    
    cutoff = 1.5
    bulk_nneighbors = 3
    i_list, j_list = neighbor_list("ij", current_md_structure, cutoff)
    counts = np.bincount(i_list, minlength=len(current_md_structure))
    edgeinds = list(np.where(counts < bulk_nneighbors)[0])
    
    allinds = np.unique(leftinds + rightinds + edgeinds)
    current_md_structure.set_constraint(FixAtoms(mask=allinds))
    current_md_structure.calc = calc # Use the Tersoff calculator defined earlier

    # 2. Run MD
    # Use unique filename per process to avoid conflicts
    # Ensure MD_files directory exists
    os.makedirs("MD_files", exist_ok=True)
    # UPDATED: Filename now includes Nsteps so changing steps forces a re-run
    temp_xyz_file = f"MD_files/md_T{int(md_temp_K)}_L{ntile}_N{md_nsteps}.xyz"
    
    if os.path.exists(temp_xyz_file):
        # pass
        # Check if file seems valid (has content)
         if os.path.getsize(temp_xyz_file) > 100:
            pass
         else:
             print(f"File {temp_xyz_file} seemed empty, re-running.")
             MaxwellBoltzmannDistribution(current_md_structure, temperature_K=md_temp_K)
             dyn = Langevin(current_md_structure, timestep*units.fs, temperature_K=md_temp_K, friction=0.01/units.fs, logfile=None)
             dyn.attach(lambda: write(temp_xyz_file, current_md_structure, append=True), interval=md_dump)
             dyn.run(md_nsteps)
    else:
        MaxwellBoltzmannDistribution(current_md_structure, temperature_K=md_temp_K)
        dyn = Langevin(current_md_structure, timestep*units.fs, temperature_K=md_temp_K, friction=0.01/units.fs, logfile=None)
        dyn.attach(lambda: write(temp_xyz_file, current_md_structure, append=True), interval=md_dump)
        
        # Run simulation
        dyn.run(md_nsteps)
    
    # 3. Analyze Trajectory
    try:
        traj = read(temp_xyz_file, index=":")
        # Check for empty trajectory
        if len(traj) == 0:
             return {E: (np.nan, np.nan) for E in E_fermi_list}
    except Exception as e:
        print(f"Error reading {temp_xyz_file}: {e}")
        return {E: (np.nan, np.nan) for E in E_fermi_list}
        
    start_analysis_frame = int(len(traj)*0.25)
    traj_analysis = traj[start_analysis_frame:]
    
    if len(traj_analysis) == 0:
        return {E: (np.nan, np.nan) for E in E_fermi_list}

    # Initialize dictionary to hold list of Transmission values for each energy
    trans_samples_dict = {E: [] for E in E_fermi_list}
    
    for frame in traj_analysis:
        pos = frame.positions
        x_sort_args = np.lexsort((pos[:, 0], pos[:, 2]))
        
        if len(x_sort_args) > 10:
             x_sort_args[[9, 10]] = x_sort_args[[10, 9]]
             
        pos_sorted = pos[x_sort_args]
        
        H_full_frame = hamiltonian(pos_sorted)
        n_lead_atoms = 20
        idx_D = np.arange(n_lead_atoms, len(pos_sorted)-n_lead_atoms)
        H_D_frame = H_full_frame[np.ix_(idx_D, idx_D)]
        
        # Calculate T for each energy
        for E in E_fermi_list:
            sig_L, sig_R = self_energies[E]
            T = calculate_T_at_energy(H_D_frame, sig_L, sig_R, E)
            trans_samples_dict[E].append(T)
    

    R_quantum = 12906.0
    
    final_results = {}
    for E in E_fermi_list:
        trans_samples = trans_samples_dict[E]
        
        # Method 1: <R> = 1 / <T>
        avg_transmission = np.mean(trans_samples)
        if avg_transmission < 1e-12:
            res_from_avg_T = np.nan
        else:
            res_from_avg_T = (1.0 / avg_transmission) * R_quantum
            
        # Method 2: <R> = <1/T>
        R_samples = resistance(trans_samples) * R_quantum
        res_avg_instant = np.mean(R_samples)
        
        final_results[E] = (res_from_avg_T, res_avg_instant)
    
    return final_results

# --- Smart Data Loading & Execution ---

output_dir = "Defected_tranmission_data/Resistance_Scaling"
os.makedirs(output_dir, exist_ok=True)

# Master database of loaded results: results_db[E][length] = (R1, R2)
results_db = {E: {} for E in E_fermi_list}

# Load existing files for each energy
for E in E_fermi_list:
    filename = f"Resistance_vs_Length_T{int(md_temp_K)}_E{E}_Nsteps{md_nsteps}_Kind{kind}_W{n}.csv"
    filepath = os.path.join(output_dir, filename)
    
    if os.path.exists(filepath):
        try:
            existing_data = np.loadtxt(filepath, delimiter=",", comments="#")
            if existing_data.size > 0:
                if existing_data.ndim == 1:
                    existing_data = existing_data.reshape(1, -1)
                for row in existing_data:
                    results_db[E][int(row[0])] = (row[1], row[2])
            print(f"Loaded {len(results_db[E])} points for E={E}")
        except Exception:
            pass

# Determine lengths missing for ANY energy
ntiles_to_run = set()
for t in ntiles_list:
    for E in E_fermi_list:
        if t not in results_db[E]:
            ntiles_to_run.add(t)
            break 
            
ntiles_to_run = sorted(list(ntiles_to_run))

if ntiles_to_run:
    print(f"Calculating for lengths: {ntiles_to_run} across all energies")
    
    # Run simulations
    new_results_list = Parallel(n_jobs=-1)(
        delayed(run_simulation_for_length)(ntile) for ntile in tqdm(ntiles_to_run, desc="Processing Lengths")
    )
    
    # Update DB
    for t, res_dict in zip(ntiles_to_run, new_results_list):
        for E, val in res_dict.items():
            results_db[E][t] = val
else:
    print("All requested data available.")

# Save updated files and Plot
plt.figure(figsize=(12, 8))
colors = plt.cm.plasma(np.linspace(0, 1, len(E_fermi_list)))

for i, E in enumerate(E_fermi_list):
    # Save
    filename = f"Resistance_vs_Length_T{int(md_temp_K)}_E{E}_Nsteps{md_nsteps}_Kind{kind}_W{n}.csv"
    filepath = os.path.join(output_dir, filename)
    
    lengths = sorted(results_db[E].keys())
    data_save = []
    
    # Arrays for plotting (subset requested)
    plot_x = []
    plot_y = []
    
    for L in lengths:
        val = results_db[E][L]
        data_save.append([L, val[0], val[1]])
        if L in ntiles_list:
            plot_x.append(L)
            plot_y.append(val[1]) # 1/<T> = val[0], <1/T> = val[1]
            
    np.savetxt(filepath, data_save, delimiter=",", 
               header=f"Length,R_Method1,R_Method2\nParams: T={md_temp_K}, E={E}")
    
    # Plot Method 1
    plt.plot(plot_x, plot_y, '-o', color=colors[i], label=f'E={E} eV')

plt.xlabel('Length (Unit Cells)')
plt.ylabel(r'Resistance $\langle R \rangle = \langle R_Q /  T \rangle$ ($\Omega$)')
plt.title(f'Resistance Scaling at T={md_temp_K}K for different Fermi Energies')
#plt.yscale('log')
plt.ylim(0,10000000000)
plt.legend()
plt.grid(True, which="both", ls="-")
plt.show()

In [ ]:
# --- LDOS Analysis for specific Lengths and Energy ---
def plot_ldos_for_length(ntile, target_E):
    # 1. Locate the correct MD file
    filename = f"MD_files/md_T{int(md_temp_K)}_L{ntile}_N{md_nsteps}.xyz"
    
    if not os.path.exists(filename):
        print(f"XYZ file not found for L={ntile}: {filename}")
        print("Required simulation may not have run yet.")
        return

    print(f"Analyzing LDOS for L={ntile}, E={target_E} eV from {filename}...")
    
    # 2. Read the thermalized structure (Last frame)
    try:
        traj = read(filename, index=-1) # Take last frame
    except Exception as e:
        print(f"Error reading file: {e}")
        return

    pos = traj.positions
    
    # 3. Sort Atoms (Crucial for matrix mapping)
    x_sort_args = np.lexsort((pos[:, 0], pos[:, 2]))
    if len(x_sort_args) > 10:
        x_sort_args[[9, 10]] = x_sort_args[[10, 9]]
    pos_sorted = pos[x_sort_args]
    
    # 4. Construct Hamiltonian for this frame
    H_sys = hamiltonian(pos_sorted)
    
    # 5. Get Self-Energies (Use existing dictionary or compute)
    if target_E in self_energies:
        sL, sR = self_energies[target_E]
    else:
        print(f"Computing self-energy for E={target_E}...")
        z_fermi = target_E + 1j * eta
        sL, _, _, _ = self_energy_decimation(z_fermi, H_L, t_L, iterations=100)
        sR, _, _, _ = self_energy_decimation(z_fermi, H_R, t_R, iterations=100)
    
    # 6. Build Green's Function
    size = H_sys.shape[0]
    n_lead_L = sL.shape[0]
    n_lead_R = sR.shape[0]
    
    # Effective Hamiltonian with Self-Energies on the leads
    Sigma_matrix = np.zeros((size, size), dtype=complex)
    Sigma_matrix[:n_lead_L, :n_lead_L] = sL
    Sigma_matrix[-n_lead_R:, -n_lead_R:] = sR
    
    z_plus = target_E + 1j * eta
    G_ret = np.linalg.inv(z_plus * np.eye(size) - H_sys - Sigma_matrix)
    
    # 7. Calculate LDOS
    # LDOS(r, E) = -1/pi * Im[ G(r,r,E) ]
    ldos = -1.0 / np.pi * np.imag(np.diag(G_ret))
    
    # 8. Plot
    plt.figure(figsize=(12, 3))
    # Scatter plot: Z vs X (top down view), colored by LDOS
    sc = plt.scatter(pos_sorted[:, 2], pos_sorted[:, 0], c=ldos, cmap='inferno', s=100, alpha=0.9, edgecolors='none', vmin=0, vmax=0.25)
    plt.colorbar(sc, label=r'LDOS (states/eV)')
    plt.title(f'Local Density of States (LDOS) at E={target_E} eV\nLength L={ntile} (Disordered Snapshot)')
    plt.xlabel('Transport Direction Z ($\AA$)')
    plt.ylabel('Width Direction X ($\AA$)')
    plt.axis('equal') # Preserve aspect ratio to see true geometry
    plt.tight_layout()
    plt.show()

# Run the plotting for specific requested lengths
requested_length = 20
target_energies = [0.1, 3]

for E in target_energies:
    plot_ldos_for_length(requested_length, E)

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Parameters for Animation
anim_length = 20
anim_energies = [0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 3, 4, 5, 6, 7, 8]

# 1. Locate and Load Structure for L=20
filename = f"MD_files/md_T{int(md_temp_K)}_L{anim_length}_N{md_nsteps}.xyz"

if not os.path.exists(filename):
    print(f"File not found: {filename}")
else:
    print(f"Generating animation for L={anim_length} across energies: {anim_energies}")
    
    # Read the thermalized structure (Last frame)
    try:
        traj = read(filename, index=-1)
        pos = traj.positions
        
        # Sort Atoms
        x_sort_args = np.lexsort((pos[:, 0], pos[:, 2]))
        if len(x_sort_args) > 10:
            x_sort_args[[9, 10]] = x_sort_args[[10, 9]]
        pos_sorted = pos[x_sort_args]
        
        # Construct Hamiltonian (Geometry is constant for all energies in this snapshot)
        H_sys = hamiltonian(pos_sorted)
        size = H_sys.shape[0]
        
        # Pre-calculate/Check Self-Energies
        # Ensure we have the self-energies for all requested animation energy points
        for E in anim_energies:
            if E not in self_energies:
                print(f"Computing missing self-energy for E={E}...")
                z_fermi = E + 1j * eta
                sL_calc, _, _, _ = self_energy_decimation(z_fermi, H_L, t_L, iterations=100)
                sR_calc, _, _, _ = self_energy_decimation(z_fermi, H_R, t_R, iterations=100)
                self_energies[E] = (sL_calc, sR_calc)

        # Setup Plot
        fig, ax = plt.subplots(figsize=(12, 4))
        
        # Initialize scatter with dummy data
        # We use a mutable scatter object
        # Initialize with fixed color limits
        sc = ax.scatter(pos_sorted[:, 2], pos_sorted[:, 0], c=np.zeros(len(pos_sorted)), 
                        cmap='inferno', s=100, alpha=0.9, edgecolors='none', vmin=0, vmax=0.25)
        
        cbar = plt.colorbar(sc, label=r'LDOS (states/eV)')
        
        ax.set_xlabel('Transport Direction Z ($\AA$)')
        ax.set_ylabel('Width Direction X ($\AA$)')
        ax.axis('equal')
        
        def update(frame_idx):
            target_E = anim_energies[frame_idx]
            
            # Retrieve Self-Energies
            sL, sR = self_energies[target_E]
            n_lead_L = sL.shape[0]
            n_lead_R = sR.shape[0]
            
            # Build Sigma Matrix
            Sigma_matrix = np.zeros((size, size), dtype=complex)
            Sigma_matrix[:n_lead_L, :n_lead_L] = sL
            Sigma_matrix[-n_lead_R:, -n_lead_R:] = sR
            
            # Calculate Green's Function
            z_plus = target_E + 1j * eta
            G_ret = np.linalg.inv(z_plus * np.eye(size) - H_sys - Sigma_matrix)
            
            # Calculate LDOS
            ldos = -1.0 / np.pi * np.imag(np.diag(G_ret))
            
            # Update Plot Data
            sc.set_array(ldos)
            # sc.set_clim(vmin=np.min(ldos), vmax=np.max(ldos)) # Disable auto-scale
            
            ax.set_title(f'Local Density of States (LDOS)\nLength L={anim_length} | Energy E={target_E} eV')
            return sc,

        # Create Animation
        anim = FuncAnimation(fig, update, frames=len(anim_energies), interval=1000, blit=False)
        plt.close(fig) # Prevent static plot compilation output
        
        # Save Animation to GIF
        output_file = f"LDOS_animation_L{anim_length}.gif"
        print(f"Saving animation to {output_file} ...")
        anim.save(output_file, writer='pillow', fps=2)
        print("Done saving.")
        
        # Display as HTML (JavaScript)
        display(HTML(anim.to_jshtml()))
        
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
# --- Plotting Transmission vs Length & Temperature for T = 0, 50, 100, 150 ---
import numpy as np
import matplotlib.pyplot as plt
import os

# Parameters
temps = [0, 50, 100, 150]
E = 0.0
md_nsteps = 1000
kind = "armchair"
width = 5
output_dir = "Defected_tranmission_data/Resistance_Scaling"
R_quantum = 12906.0

# Data Storage
data_all = {} # {T: (lengths, T_avg)}

# 1. Load Data
print("Loading data...")
for T in temps:
    filename = f"Resistance_vs_Length_T{T}_E{E}_Nsteps{md_nsteps}_Kind{kind}_W{width}.csv"
    filepath = os.path.join(output_dir, filename)
    
    if os.path.exists(filepath):
        try:
            data = np.loadtxt(filepath, delimiter=",", comments="#")
            if data.size > 0:
                if data.ndim == 1: data = data.reshape(1, -1)
                # Sort by length
                data = data[data[:, 0].argsort()]
                lengths = data[:, 0]
                
                # Column 1 is R_Method1 = (1 / <T>) * R_quantum
                # So <T> = R_quantum / R_Method1
                r_method1 = data[:, 1]
                
                # Avoid division by zero if R is zero (unlikely but safe to check)
                # If R=nan, T=nan
                T_avg = np.divide(R_quantum, r_method1, out=np.zeros_like(r_method1), where=r_method1!=0)
                
                data_all[T] = (lengths, T_avg)
                print(f"Loaded T={T} K: {len(lengths)} data points.")
        except Exception as e:
            print(f"Error loading {filename}: {e}")
    else:
        print(f"File not found: {filename}")

# 2. Plot Transmission vs Length (Linear Scale Analysis)
plt.figure(figsize=(10, 6))
colors = plt.cm.viridis(np.linspace(0, 1, len(temps)))

for i, T in enumerate(temps):
    if T in data_all:
        lengths, T_val = data_all[T]
        plt.plot(lengths, T_val, '-o', label=f'T={T} K', color=colors[i])

plt.title(f"Transmission vs Length at Different Temperatures (E={E} eV)")
plt.xlabel("Length (Unit Cells)")
plt.ylabel(r"Average Transmission $\langle T \rangle$ ($G_0$)")
# plt.yscale('log')
plt.legend()
plt.grid(True, which="both", ls="-", alpha=0.5)
plt.show()

# 3. Plot Transmission vs Temperature for selected Lengths
selected_lengths = [10, 20, 30, 40, 50]
plt.figure(figsize=(10, 6))

for L in selected_lengths:
    t_list = []
    T_val_list = []
    for T in temps:
        if T in data_all:
            lengths, T_val = data_all[T]
            # Find closest length
            idx = np.where(np.isclose(lengths, L))[0]
            if len(idx) > 0:
                t_list.append(T)
                T_val_list.append(T_val[idx[0]])
    if t_list:
        plt.plot(t_list, T_val_list, '-o', label=f'L={L}')

plt.title(f"Transmission vs Temperature for Fixed Lengths (E={E} eV)")
plt.xlabel("Temperature (K)")
plt.ylabel(r"Average Transmission $\langle T \rangle$ ($G_0$)")
# plt.yscale('log')
plt.legend()
plt.grid(True, which="both", ls="-", alpha=0.5)
plt.show()